# Pokemon Web Scraping

In a previous project, Microsoft SQL Server was used to create a table of Pokemon from generations 1-3 and their stats to query the data within the table. That SQL script manually created a table and entered all of the data of Pokemon, however this is a tedious and time consuming method of collecting data, especially if new Pokemon need to be added periodically.

Rather than using SQL to manually fill in data, this Python script will be used to scrape all of the necessary data from the Pokemon online database, insert it into a dataframe, then create a .csv file to be used in other software.

https://pokemondb.net/pokedex/all

--------------------------------

Import the necessary libraries and pull the html information from the web page

In [4]:
#Import Libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import datetime

In [5]:
#Connect to Website
url = 'https://pokemondb.net/pokedex/all'

#Obtain User Agent
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}

#Use requests library to get the page using the url and headers
page = requests.get(url, headers = headers)

#Pull Content from the Page as html
soup1 = BeautifulSoup(page.content, 'html.parser')

#Improve Readability
soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

--------------------------

Find the necessary information from the web page and insert it into a dataframe

In [6]:
#Find the full table from the web page
table = soup2.find('table', {'id':'pokedex'})

In [7]:
#Find the column names
columns = table.find_all('th')

column_names = [title.text.strip() for title in columns]

print(column_names)

['#', 'Name', 'Type', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']


In [8]:
#Create a pandas dataframe and insert the column names
df = pd.DataFrame(columns = column_names)
df

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed


In [9]:
#Find the rows of data
data = table.find_all('tr')

for row in data[1:]:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]
    
    #Insert the rows into the dataframe 
    length = len(df)
    df.loc[length] = individual_row_data

In [10]:
df

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0001,Bulbasaur,Grass\n \n\n\n Poison,318,45,49,49,65,65,45
1,0002,Ivysaur,Grass\n \n\n\n Poison,405,60,62,63,80,80,60
2,0003,Venusaur,Grass\n \n\n\n Poison,525,80,82,83,100,100,80
3,0003,Venusaur\n \n\n\n Mega Venu...,Grass\n \n\n\n Poison,625,80,100,123,122,120,80
4,0004,Charmander,Fire,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1189,1006,Iron Valiant,Fairy\n \n\n\n Fighting,590,74,130,90,120,60,116
1190,1007,Koraidon,Fighting\n \n\n\n Dragon,670,100,135,115,85,100,135
1191,1008,Miraidon,Electric\n \n\n\n Dragon,670,100,85,100,135,115,135
1192,1009,Walking Wake,Water\n \n\n\n Dragon,590,99,83,91,125,83,109


-----------------

Clean the Dataframe

In [11]:
#Remove unnecessary spaces/characters in the Name and Type Columns
#In these cases, the excess characters are marking sub-labels and special formatting, so they will will be replaced
#with a comma to preserve the separation of information. This with make further separtion in later steps easier.

df['Name'] = df['Name'].replace('\n          \n\n\n          ', ',', regex = True)
df['Type'] = df['Type'].replace('\n          \n\n\n          ', ',', regex = True)

df

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0001,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45
1,0002,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60
2,0003,Venusaur,"Grass, Poison",525,80,82,83,100,100,80
3,0003,"Venusaur, Mega Venusaur","Grass, Poison",625,80,100,123,122,120,80
4,0004,Charmander,Fire,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...
1189,1006,Iron Valiant,"Fairy, Fighting",590,74,130,90,120,60,116
1190,1007,Koraidon,"Fighting, Dragon",670,100,135,115,85,100,135
1191,1008,Miraidon,"Electric, Dragon",670,100,85,100,135,115,135
1192,1009,Walking Wake,"Water, Dragon",590,99,83,91,125,83,109


In [12]:
#Using the commas inserted in the previous step, the typ column will be split to create a Type 1 and Type 2 column for
#multi-type Pokemon.

df[['Type 1', 'Type 2']] = df['Type'].str.split(',', 1, expand = True)

df

C:\Users\sadie\AppData\Local\Temp\ipykernel_39184\3964475195.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['Type 1', 'Type 2']] = df['Type'].str.split(',', 1, expand = True)


,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
0,0001,Bulbasaur,"Grass, Poison",318,45,49,49,65,65,45,Grass,Poison
1,0002,Ivysaur,"Grass, Poison",405,60,62,63,80,80,60,Grass,Poison
2,0003,Venusaur,"Grass, Poison",525,80,82,83,100,100,80,Grass,Poison
3,0003,"Venusaur, Mega Venusaur","Grass, Poison",625,80,100,123,122,120,80,Grass,Poison
4,0004,Charmander,Fire,309,39,52,43,60,50,65,Fire,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1189,1006,Iron Valiant,"Fairy, Fighting",590,74,130,90,120,60,116,Fairy,Fighting
1190,1007,Koraidon,"Fighting, Dragon",670,100,135,115,85,100,135,Fighting,Dragon
1191,1008,Miraidon,"Electric, Dragon",670,100,85,100,135,115,135,Electric,Dragon
1192,1009,Walking Wake,"Water, Dragon",590,99,83,91,125,83,109,Water,Dragon


In [13]:
#Now that the Type column has been split into two new columns, it will be dropped.

df = df.drop(columns = 'Type')

df

,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
0,0001,Bulbasaur,318,45,49,49,65,65,45,Grass,Poison
1,0002,Ivysaur,405,60,62,63,80,80,60,Grass,Poison
2,0003,Venusaur,525,80,82,83,100,100,80,Grass,Poison
3,0003,"Venusaur, Mega Venusaur",625,80,100,123,122,120,80,Grass,Poison
4,0004,Charmander,309,39,52,43,60,50,65,Fire,None
...,...,...,...,...,...,...,...,...,...,...,...
1189,1006,Iron Valiant,590,74,130,90,120,60,116,Fairy,Fighting
1190,1007,Koraidon,670,100,135,115,85,100,135,Fighting,Dragon
1191,1008,Miraidon,670,100,85,100,135,115,135,Electric,Dragon
1192,1009,Walking Wake,590,99,83,91,125,83,109,Water,Dragon


In [14]:
#In the Name column, the only entires will commas are those with "Mega" evolutions. These are unique to specific versions
#of specific games and have been deemed unnecessary, so these rows will be dropped.

df = df[df['Name'].str.contains(',') == False]

df

,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
0,0001,Bulbasaur,318,45,49,49,65,65,45,Grass,Poison
1,0002,Ivysaur,405,60,62,63,80,80,60,Grass,Poison
2,0003,Venusaur,525,80,82,83,100,100,80,Grass,Poison
4,0004,Charmander,309,39,52,43,60,50,65,Fire,None
5,0005,Charmeleon,405,58,64,58,80,65,80,Fire,None
...,...,...,...,...,...,...,...,...,...,...,...
1189,1006,Iron Valiant,590,74,130,90,120,60,116,Fairy,Fighting
1190,1007,Koraidon,670,100,135,115,85,100,135,Fighting,Dragon
1191,1008,Miraidon,670,100,85,100,135,115,135,Electric,Dragon
1192,1009,Walking Wake,590,99,83,91,125,83,109,Water,Dragon


In [16]:
#Since rows were dropped, the datframe's index will be reset

df = df.reset_index(drop = True)

df

,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1,Type 2
0,0001,Bulbasaur,318,45,49,49,65,65,45,Grass,Poison
1,0002,Ivysaur,405,60,62,63,80,80,60,Grass,Poison
2,0003,Venusaur,525,80,82,83,100,100,80,Grass,Poison
3,0004,Charmander,309,39,52,43,60,50,65,Fire,None
4,0005,Charmeleon,405,58,64,58,80,65,80,Fire,None
...,...,...,...,...,...,...,...,...,...,...,...
967,1006,Iron Valiant,590,74,130,90,120,60,116,Fairy,Fighting
968,1007,Koraidon,670,100,135,115,85,100,135,Fighting,Dragon
969,1008,Miraidon,670,100,85,100,135,115,135,Electric,Dragon
970,1009,Walking Wake,590,99,83,91,125,83,109,Water,Dragon


-------------------

Save the dataframe as a .csv file

In [15]:
df.to_csv(r'C:\Users\sadie\Python Practice\PokemonWebScraping.csv', index = False)

---------------------

Pokemon games are still being released fairly regulary, so a function of all the above steps will be created and set to run twice a year to check for any new Pokemon added to the database.

In [7]:
def pokemon_check():
    #Import Libraries
    from bs4 import BeautifulSoup
    import pandas as pd
    import requests
    import time
    import datetime
    
    #Gather Web Page html
    url = 'https://pokemondb.net/pokedex/all'
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
    page = requests.get(url, headers = headers)
    soup1 = BeautifulSoup(page.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
    
    #Pull Table and Column Headers
    table = soup2.find('table', {'id':'pokedex'})
    columns = table.find_all('th')
    column_names = [title.text.strip() for title in columns]
    
    #Create Dataframe and Insert Columns Headers
    df = pd.DataFrame(columns = column_names)
    
    #Pull Data and Insert into Dataframe
    data = table.find_all('tr')
    for row in data[1:]:
        row_data = row.find_all('td')
        individual_row_data = [data.text.strip() for data in row_data] 
        length = len(df)
        df.loc[length] = individual_row_data
        
    #Clean Data
    df['Name'] = df['Name'].replace('\n          \n\n\n          ', ',', regex = True)
    df['Type'] = df['Type'].replace('\n          \n\n\n          ', ',', regex = True)
    df[['Type 1', 'Type 2']] = df['Type'].str.split(',', 1, expand = True)
    df = df.drop(columns = 'Type')
    df = df[df['Name'].str.contains(',') == False]
    df = df.reset_index(drop = True)
    
    #Save the Datframe
    df.to_csv(r'C:\Users\sadie\Python Practice\PokemonWebScraping.csv', index = False)

In [9]:
#Automate to Perform the Function at Set Intervals
while(True):
    pokemon_check()
    time.sleep(60*60*24*183) #Run the functions every 183 days (twice a year)

        #          Name Total   HP Attack Defense Sp. Atk Sp. Def Speed  \
0    0001     Bulbasaur   318   45     49      49      65      65    45   
1    0002       Ivysaur   405   60     62      63      80      80    60   
2    0003      Venusaur   525   80     82      83     100     100    80   
3    0004    Charmander   309   39     52      43      60      50    65   
4    0005    Charmeleon   405   58     64      58      80      65    80   
..    ...           ...   ...  ...    ...     ...     ...     ...   ...   
967  1006  Iron Valiant   590   74    130      90     120      60   116   
968  1007      Koraidon   670  100    135     115      85     100   135   
969  1008      Miraidon   670  100     85     100     135     115   135   
970  1009  Walking Wake   590   99     83      91     125      83   109   
971  1010   Iron Leaves   590   90    130      88      70     108   104   

       Type 1     Type 2  
0       Grass     Poison  
1       Grass     Poison  
2       Grass     

C:\Users\sadie\AppData\Local\Temp\ipykernel_37300\1330683965.py:35: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['Type 1', 'Type 2']] = df['Type'].str.split(',', 1, expand = True)


NameError: name 'time' is not defined